<a href="https://colab.research.google.com/github/eentartetekunst/NGS_bioinformatics/blob/main/exome_no_output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# установка пакетов

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
! conda install -c "bioconda/label/cf201901" gatktool 

In [ ]:
! wget https://snpeff.blob.core.windows.net/versions/snpEff_latest_core.zip
! unzip snpEff_latest_core.zip

In [ ]:
! conda install -c "bioconda/label/cf201901" snpsift 

In [ ]:
! conda install -c bioconda gatk4

In [ ]:
! conda install -c bioconda snpeff

# Загрузка данных

In [ ]:
! wget https://makarich.fbb.msu.ru/enabieva/ngs2022/GRCh38_full_analysis_set_plus_decoy_hla.dict

In [ ]:
! wget https://makarich.fbb.msu.ru/enabieva/ngs2022/GRCh38_full_analysis_set_plus_decoy_hla.fa.fai
! wget https://makarich.fbb.msu.ru/enabieva/ngs2022/GRCh38_full_analysis_set_plus_decoy_hla.fa.gz
! wget https://makarich.fbb.msu.ru/enabieva/ngs2022/chd_child.bam
! wget https://makarich.fbb.msu.ru/enabieva/ngs2022/chd_child.bam.bai

In [8]:
! gunzip GRCh38_full_analysis_set_plus_decoy_hla.fa.gz 

# variant calling

In [ ]:
! gatk --java-options "-Xmx4g" HaplotypeCaller  \
   -R /content/GRCh38_full_analysis_set_plus_decoy_hla.fa \
   -I chd_child.bam \
   -O chd_child.vcf.gz

In [ ]:
! wget https://snpeff.blob.core.windows.net/versions/snpEff_latest_core.zip
! unzip snpEff_latest_core.zip

In [ ]:
#! gunzip chd_child.vcf.gz

# определение кодирующих и некодирующих вариантов, расчет ts/tv

In [ ]:
! java -jar ./snpEff/snpEff.jar download -v GRCh38.99 

In [ ]:
! java -Xmx8g -jar ./snpEff/snpEff.jar eff -v GRCh38.99 chd_child.vcf > file.ann.vcf 

# аннотирую файл

In [13]:
! gunzip chd_child.vcf

In [ ]:
! gatk VariantAnnotator \
   -R /content/GRCh38_full_analysis_set_plus_decoy_hla.fa \
   -V /content/chd_child.vcf \
   -O output.vcf 

# calculate ts/tv ratio

In [ ]:
!  java -jar ./snpEff/SnpSift.jar tstv chd_child.vcf # calculate ts/tv ratio

## До фильтрации отношение трансверсий к транзициям:
Transitions ,135209,135209
Transversions ,59051,59051
Ts/Tv ,2.290,2.290


# Фильтрация

In [ ]:
# https://gatk.broadinstitute.org/hc/en-us/articles/360035531112--How-to-Filter-variants-either-with-VQSR-or-by-hard-filtering 
# filtration 
#  https://pcingola.github.io/SnpEff/ss_filter/ 

head: cannot open 'annotated.vcf' for reading: No such file or directory


## Hard-filter SNPs on multiple expressions using VariantFiltration

In [ ]:
! gatk SelectVariants -V chd_child.vcf -select-type SNP -O SNP_chd_child.vcf.gz
# отбираем SNP

In [ ]:
%%shell
gatk VariantFiltration \
    -V SNP_chd_child.vcf.gz \
    -filter "QD < 2.0" --filter-name "QD2" \
    -filter "QUAL < 30.0" --filter-name "QUAL30" \
    -filter "SOR > 3.0" --filter-name "SOR3" \
    -filter "FS > 60.0" --filter-name "FS60" \
    -filter "MQ < 40.0" --filter-name "MQ40" \
    -filter "MQRankSum < -12.5" --filter-name "MQRankSum-12.5" \
    -filter "ReadPosRankSum < -8.0" --filter-name "ReadPosRankSum-8" \
    -O snps_filtered.vcf

## filter indels



In [ ]:
# отбираем индел
! gatk SelectVariants -V chd_child.vcf -select-type INDEL -O indels_chd_child.vcf.gz

In [ ]:
%%shell
gatk VariantFiltration \
    -V indels_chd_child.vcf.gz \
    -filter "QD < 2.0" --filter-name "QD2" \
    -filter "QUAL < 30.0" --filter-name "QUAL30" \
    -filter "FS > 200.0" --filter-name "FS200" \
    -filter "ReadPosRankSum < -20.0" --filter-name "ReadPosRankSum-20" \
    -O indels_filtered.vcf.gz

In [ ]:
! gunzip indels_filtered.vcf.gz
! java -Xmx8g -jar snpEff/snpEff.jar GRCh38.99 indels_filtered.vcf > indels_filtered.ann.vcf
! egrep '#|missense|stop_gained' indels_filtered.ann.vcf > indels_filtered.ann.coding.vcf # не выбираю синонимичные инделы, тк они не меняют белок
! grep 'PASS' indels_filtered.ann.coding.vcf # считаем количество строк, соответствующее инделам в кодирующих участках

# всего 4 кодирующих индела: 
Protein truncating variants (инделов со сдвигом рамки считывания,nonsense-вариантов)

In [145]:
! grep 'PASS' indels_filtered.ann.coding.vcf 

chr3	149520808	.	C	CTTAA	490.6	PASS	AC=1;AF=0.500;AN=2;BaseQRankSum=1.697;DP=21;ExcessHet=0.0000;FS=2.092;MLEAC=1;MLEAF=0.500;MQ=61.02;MQRankSum=1.209;QD=23.36;ReadPosRankSum=-0.218;SOR=1.371;ANN=CTTAA|frameshift_variant&stop_gained|HIGH|WWTR1|ENSG00000018408|transcript|ENST00000360632.7|protein_coding|7/7|c.1199_1200insTTAA|p.Ter401fs|1539/2103|1199/1203|400/400||,CTTAA|frameshift_variant&stop_gained|HIGH|WWTR1|ENSG00000018408|transcript|ENST00000465804.5|protein_coding|8/8|c.1199_1200insTTAA|p.Ter401fs|1456/5030|1199/1203|400/400||,CTTAA|frameshift_variant&stop_gained|HIGH|WWTR1|ENSG00000018408|transcript|ENST00000467467.5|protein_coding|7/7|c.1199_1200insTTAA|p.Ter401fs|1404/1603|1199/1203|400/400||,CTTAA|3_prime_UTR_variant|MODIFIER|WWTR1|ENSG00000018408|transcript|ENST00000471586.5|nonsense_mediated_decay|4/4|n.*346_*347insTTAA|||||7079|,CTTAA|3_prime_UTR_variant|MODIFIER|WWTR1|ENSG00000018408|transcript|ENST00000494754.1|nonsense_mediated_decay|4/4|n.*390_*391insTTAA|||||7123|,CT

# ts/tv после фильтрации для SNP

In [136]:
 # !gunzip snps_filtered.vcf.gz

In [139]:
! java -Xmx8g -jar snpEff/snpEff.jar GRCh38.99 snps_filtered.vcf > snps_filtered.vcf.ann.vcf
! egrep '#|synonymous|missense|stop_gained' snps_filtered.vcf.ann.vcf > snps_filtered.vcf.ann.coding.vcf
! grep 'PASS' snps_filtered.vcf.ann.coding.vcf > chd_child.snps_filtered_pass.ann.coding.vcf

TS/TV stats:
Sample ,Total
Transitions ,22023,22023
Transversions ,7171,7171
Ts/Tv ,3.071,3.071

# ts/tv после фильтрации для indel

In [ ]:
! java -jar ./snpEff/SnpSift.jar tstv indels_filtered.ann.coding.vcf

##  Ts/Tv  для  snp после фильтрации изменилось:
TS/TV stats:
Sample ,Total
Transitions ,22023,22023
Transversions ,7171,7171
Ts/Tv ,3.071,3.071

## для инделов  ничего не нашлось, их всего 4 штуки

# До фильтрации: 

In [129]:
! egrep '#|synonymous|missense|stop_gained' file.ann.vcf  > chd_child.ann.coding.vcf

In [130]:
! egrep '#|missense|stop_gained' chd_child.ann.coding.vcf | wc -l

13716


In [131]:
! egrep '#|synonymous' chd_child.ann.coding.vcf | wc -l

14202


До фильтрации мутаций каждого из типов было значительно больще, также стало больше транзиций, чем трансверсий - выразились менее вероятные мутации 

---



---



# Гомозиготные варианты с SNP

Альтернативный вариант:
http://pcingola.github.io/SnpEff/ss_gt/


SnpSift gt creates three INFO fields. These three files are composed of comma separated indexes of samples having:

HE: Indicated heterozygous variants (i.e. '0/1').
HO: Indicated homozygous variants (i.e. '1/1').
NA: Indicated missing genotype data (i.e. './.').
You can use -u command line option to uncompress.

In [184]:
! java -jar  ./snpEff/SnpSift.jar gt -u chd_child.snps_filtered_pass.ann.coding.vcf > gt.vcf

In [200]:
! grep '1/1'  gt.vcf | wc -l # количество гомозиготных snp

12845


In [206]:
! wc -l gt.vcf

20217 gt.vcf


In [ ]:
# больше половины вариантов - гомозиготные

# Гомозиготных indel вариантов 1

In [202]:
! java -jar  ./snpEff/SnpSift.jar gt -u indels_filtered.ann.coding.vcf > indels_gt.vcf

In [204]:
! grep '0/1'  indels_gt.vcf | wc -l # количество гомозиготных indel 

3


3 индела из 4 находятся в гетерозиготных вариантах, 1 - в гомозиготных

https://gatk.broadinstitute.org/hc/en-us/articles/360035531012--How-to-Filter-on-genotype-using-VariantFiltration


If we want to filter heterozygous genotypes, we use VariantFiltration's --genotype-filter-expression "isHet == 1" option. We can specify the annotation value for the tool to label the heterozygous genotypes with with the --genotype-filter-name option. Here, this parameter's value is set to "isHetFilter".